<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl_2/blob/main/ML_T1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tarea 1. Aprendizaje máquina.**

In [5]:
import numpy as np
import matplotlib.pyplot as plt

1. Descenso de gradiente estocástico.

Descripción.
Sea $\Omega$ el conjunto de datos $\{(x_i,y_i)\}$ dados. Seleccionemos un subconjunto $S\subset \Omega$, en principio con $\#S<<\#\Omega$.

Si $f$, nuestra función objetivo a minimizar es de la forma $f(x)=\frac{1}{\#\Omega}\sum\limits_{i=1} f_i(x)$.
Se puede considerar una aproximación de se gradiente como 
$$\nabla \hat{f}(x)=\frac{1}{\# S}\sum\limits_{i=1} \nabla f_i(x)$$

In [6]:
def grad_quadratic(theta, Z):
    '''
    Gradiente de la funcion de costo 
           sum_i (theta@x[i]-y[i])**2
    '''    
    sum=0
    for i in range(Z):
        err=np.dot([theta[0]],[Z[i][0]])
        partial0=err
        partial1=Z[i][0]*partial0
        sum+=
    return n
alpha=0.2
def SGD(Z,z_0,grad_f,batch,max_iter=10**4,stop_par=10**(-5)):
    count=0
    Theta=[]
    err=100
    while(count<max_iter):
        sample = np.random.randint(low=0, high=len(Z), size=batch)# sample 
        p = grad_f(theta=theta,Z=[Z[u] for u in sample])
        theta = theta - alpha*p
        Theta.append(theta)
    return Theta[-1]

[1, 1]


1. Descenso de gradiente estocástico.


1. Descenso de gradiente estocástico.


1. Descenso de gradiente estocástico.


In [11]:
#Evaluación de algoritmos.
#Generación de datos.
#Supongamos f:R^2->R
#Consideremos los datos X,
from sklearn import linear_model, datasets

n_samples = 500
X, y = datasets.make_regression(n_samples=n_samples,
                                n_features=1,
                                n_informative=2, 
                                noise=5,
                                random_state=0) #2)
n_outliers=100
X[:n_outliers], y[:n_outliers] = datasets.make_regression(n_samples=n_outliers,
                                n_features=1,
                                n_informative=2, 
                                noise=2,
                                random_state=61)
y=np.expand_dims(y,axis=1)
#plt.scatter(X[:],y[:], marker='.')


[[-8.92532381e+01]
 [-1.61357701e+01]
 [-2.86650691e+01]
 [-6.17386649e+00]
 [ 1.22702126e+02]
 [-1.85475139e+02]
 [ 1.04443994e+02]
 [ 1.73667697e+00]
 [ 4.60789768e+01]
 [ 1.30170364e+02]
 [-6.61244194e+01]
 [ 3.47871035e+01]
 [-3.99831307e+01]
 [ 4.05440668e+01]
 [-1.22058651e+01]
 [-1.08284356e+01]
 [ 9.22299159e+01]
 [ 4.64269047e+01]
 [-4.43873211e+01]
 [ 2.58908134e+01]
 [ 4.03962472e+00]
 [ 6.20301185e+01]
 [ 2.62065926e+01]
 [-9.13994545e+01]
 [-7.98372297e+01]
 [ 5.96588832e+00]
 [ 6.94751836e+01]
 [-4.05087642e+01]
 [ 1.24217011e+01]
 [ 5.39011450e+01]
 [ 9.51977136e+00]
 [-8.26084520e+01]
 [-6.98788959e+01]
 [-3.15464644e+01]
 [-5.22447233e+01]
 [-3.64051528e+01]
 [-2.78529562e+01]
 [-3.90425973e+00]
 [-3.47281346e+01]
 [-1.47713456e+02]
 [-4.47989856e+01]
 [ 1.00834019e+02]
 [ 5.02055762e+01]
 [ 1.30017786e+02]
 [ 4.74045830e+01]
 [-8.55533397e+01]
 [-5.14620100e+01]
 [-6.31165770e+01]
 [-7.27613304e+01]
 [ 5.21880829e+01]
 [-1.18110062e+02]
 [-1.56108927e+02]
 [ 3.5275167